In [1]:
import tensorflow as tf
import numpy as np

/Users/xuchengzhu/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/xuchengzhu/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('data_dir', './data',
    """Directory of stored data.""")
tf.app.flags.DEFINE_integer('batch_size',1,
    """Size of batch""")               
tf.app.flags.DEFINE_integer('patch_size',128,
    """Size of a data patch""")
tf.app.flags.DEFINE_integer('patch_layer',128,
    """Number of layers in data patch""")
tf.app.flags.DEFINE_integer('epochs',2000,
    """Number of epochs for training""")
tf.app.flags.DEFINE_string('log_dir', './tmp/log',
    """Directory where to write training and testing event logs """)
tf.app.flags.DEFINE_float('init_learning_rate',0.0001,
    """Initial learning rate""")
tf.app.flags.DEFINE_float('decay_factor',0.01,
    """Exponential decay learning rate factor""")
tf.app.flags.DEFINE_integer('decay_steps',100,
    """Number of epoch before applying one learning rate decay""")
tf.app.flags.DEFINE_integer('display_step',10,
    """Display and logging interval (train steps)""")
tf.app.flags.DEFINE_integer('save_interval',1,
    """Checkpoint save interval (epochs)""")
tf.app.flags.DEFINE_string('checkpoint_dir', './tmp/ckpt',
    """Directory where to write checkpoint""")
tf.app.flags.DEFINE_string('model_dir','./tmp/model',
    """Directory to save model""")
tf.app.flags.DEFINE_bool('restore_training',True,
    """Restore training from last checkpoint""")
tf.app.flags.DEFINE_float('drop_ratio',0.9,
    """Probability to drop a cropped area if the label is empty. All empty patches will be droped for 0 and accept all cropped patches if set to 1""")
tf.app.flags.DEFINE_integer('min_pixel',10,
    """Minimum non-zero pixels in the cropped label""")
tf.app.flags.DEFINE_integer('shuffle_buffer_size',5,
    """Number of elements used in shuffle buffer""")

In [5]:
# define 2d basic network structure
# tensorflow app flags

def init_uniform_dist_2d(shape):
    with tf.variable_scope('uniform_dist_initializer'):
        denominator = tf.cast((tf.reduce_prod(shape[:2]) * tf.reduce_sum(shape[2:])), tf.float32)
        lim = tf.sqrt(6. / denominator)
        return tf.random_uniform(shape, minval=-lim, maxval=lim)

def init_uniform_dist(shape):
    with tf.variable_scope('uniform_dist_initializer'):
        denominator = tf.cast((tf.reduce_prod(shape)), tf.float32)
        lim = tf.sqrt(6. / denominator)
        return tf.random_uniform(shape, minval=-lim, maxval=lim)    
    
def convolution_2d(layer_input, filter, strides, padding='VALID'):
    assert len(filter) == 4  # [filter_height, filter_width, in_channels, out_channels]
    assert len(strides) == 4  # must match input dimensions [batch, in_height, in_width, in_channels]
    assert padding in ['VALID', 'SAME']

    w = tf.Variable(initial_value=init_uniform_dist_2d(shape=filter), name='weights')
    b = tf.Variable(tf.constant(1.0, shape=[filter[-1]]), name='biases')

    return tf.nn.conv2d(layer_input, w, strides, padding) + b

def convolution_block_2d(layer_input, n_channels, out_channel,num_convolutions=1):
    x = layer_input
    for i in range(num_convolutions - 1):
        with tf.variable_scope('conv_' + str(i+1)):
            x = convolution_2d(x, [5, 5, n_channels, n_channels], [1, 1, 1, 1])
            x = tf.nn.relu(x)
    x = convolution_2d(x, [5, 5, n_channels, out_channel], [1, 1, 1, 1])
    x = x
    return tf.nn.relu(x)

def fc_2d(layer_input, in_channel, out_channel):
    x = tf.reshape(layer_input,[-1,in_channel])
    w = tf.Variable(initial_value=init_uniform_dist([in_channel,out_channel]), name='weights')# shape unknown issue
    b = tf.Variable(tf.constant(1.0, shape=[out_channel]), name='biases')
    return tf.nn.relu(tf.matmul(x,w)+b)
    
def readout_2d(layer_input, in_channel, out_channel):
    x = tf.reshape(layer_input,[-1,in_channel])
    w = tf.Variable(initial_value=init_uniform_dist([in_channel,out_channel]), name='weights')# shape unknown issue
    b = tf.Variable(tf.constant(1.0, shape=[out_channel]), name='biases')
    return tf.matmul(x,w)+b
    
def Net(input,shape, n_channel=16, out_channel=10):
    x = tf.reshape(input,[-1,shape[0],shape[1]])
    x = tf.tile(x[:,:,:,None],[1,1,1,n_channel])
    
    N = 0
    for i in range(N):
        with tf.variable_scope('conv_net_'+str(i)):
            x = convolution_block_2d(x,n_channel*(i+1),n_channel*(i+2))
    print(x.shape)
    x = tf.reshape(x,[-1,(shape[0]-4*N)*(shape[1]-4*N)*(N+1)*n_channel])
    with tf.variable_scope('fc_0'):
        x = fc_2d(x, (shape[0]-4*N)*(shape[1]-4*N)*(N+1)*n_channel, 64)
        
    with tf.variable_scope('readout'):
        x = readout_2d(x, 64, out_channel)
        
    return x

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def training():
    shape = [28,28]
    x = tf.placeholder(tf.float32, [None,28*28])
    y_ = tf.placeholder(tf.float32, [None, 10])
    y = Net(x,shape)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    
    sess = tf.Session()
    train_step = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cross_entropy)
    sess.run(tf.global_variables_initializer())
    
    for iter in range(10):
        for i in range(2000):
            batch = mnist.train.next_batch(50)
            sess.run(train_step,feed_dict={x: batch[0], y_: batch[1]})
            
            if i % 200 is 0:
                loss = sess.run(cross_entropy,feed_dict={x: batch[0], y_: batch[1]})
                print("loss function:%g"%loss)
                print("Accuracy:%g"%sess.run(accuracy,feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    


In [6]:
training()

(?, 28, 28, 16)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

loss function:1.71648
Accuracy:0.1856
loss function:0.222467
Accuracy:0.915
loss function:0.227311
Accuracy:0.9233
loss function:0.264777
Accuracy:0.9397
loss function:0.156363
Accuracy:0.9562
loss function:0.0723573
Accuracy:0.9581
loss function:0.109498
Accuracy:0.9609
loss function:0.0147846
Accuracy:0.9607
loss function:0.0936627
Accuracy:0.9612
loss function:0.104914
Accuracy:0.9651
loss function:0.255487
Accuracy:0.9627
loss function:0.0976764
Accuracy:0.9639
loss function:0.0151109
Accuracy:0.9666
loss function:0.0146838
Accuracy:0.9611
loss function:0.110932
Accuracy:0.9653
loss function:0.175423
Accuracy:0.9698
loss function:0.0811564
Accuracy:0.9667
loss function:0.0855653
Accuracy:0.9709
loss function:0.00971404
Accuracy:0.9646
loss function:0.0352633
Accuracy:0.9689
loss f